In [ ]:
from IPython.core.display import HTML
with open('../style.css') as f:
    css = f.read()
HTML(css)

In [ ]:
%load_ext nb_mypy

# Bidirectional Breadth First Search

In [ ]:
from typing import TypeVar, Callable, Generator

In [ ]:
State    = TypeVar('State')
NxtStFct = Callable[[State], set[State]]

The following forward declarations are needed to typecheck the function `search`.

In [ ]:
def combinePaths(state: State, ParentA: dict[State, State], ParentB: dict[State, State]) -> list[State]:
    return None # type: ignore

In [ ]:
def bfs_one_step(Frontier: set[State], 
                 ParentA: dict[State, State], 
                 ParentB: dict[State, State], 
                 next_states: NxtStFct, 
                 flip: bool) -> Generator[list[State], None, None]:
    return None # type: ignore

The function `search` takes three arguments to solve a *search problem*:
- `start` is the *start state* of the search problem,
- `goal` is the *goal state*, and
- `next_states` is a function with signature $\texttt{next_states}:Q \rightarrow 2^Q$, where $Q$ is the set of states.
   For every state $s \in Q$, $\texttt{next_states}(s)$ is the set of states that can be reached from $s$ in one step.

If successful, `search` returns a path from `start` to `goal` that is a solution of the search problem
$$ \langle Q, \texttt{next_states}, \texttt{start}, \texttt{goal} \rangle. $$
The implementation of `search` uses *bidirectional breadth first search* to find a path from `start` to `goal`.

The old version of `search` shown below was implemented using *copy and paste*.  
I wanted to improve on this implementation.  Judge for yourself whether I have succeeded.
```
def search(start: State, goal: State, next_states: NxtStFct):        
    FrontierA = { start }
    ParentA   = { start: start }
    FrontierB = { goal }
    ParentB   = { goal: goal } 
    while FrontierA and FrontierB:
        NewFrontier = set()
        for s in FrontierA:
            for ns in next_states(s):
                if ns not in ParentA:
                    NewFrontier |= { ns }
                    ParentA[ns]  = s
                    if ns in ParentB:
                        return combinePaths(ns, ParentA, ParentB)
        FrontierA   = NewFrontier
        NewFrontier = set()
        for s in FrontierB:
            for ns in next_states(s):
                if ns not in ParentB:
                    NewFrontier |= { ns }
                    ParentB[ns]  = s
                    if ns in ParentA:
                        return combinePaths(ns, ParentA, ParentB)
        FrontierB = NewFrontier
```
Here is the new version.

In [ ]:
def search(start: State, goal: State, next_states: NxtStFct) -> list[State] | None:        
    FrontierA = { start }
    ParentA   = { start: start }
    FrontierB = { goal }
    ParentB   = { goal: goal } 
    while FrontierA and FrontierB:
        for Path in bfs_one_step(FrontierA, ParentA, ParentB, next_states, False):
            return Path
        for Path in bfs_one_step(FrontierB, ParentB, ParentA, next_states, True):
            return Path
    return None

In [ ]:
def bfs_one_step(Frontier: set[State], 
                 ParentA: dict[State, State], 
                 ParentB: dict[State, State], 
                 next_states: NxtStFct, 
                 flip: bool
                ) -> Generator[list[State], None, None]:
    NewFrontier = set()
    for s in Frontier:
        for ns in next_states(s):
            if ns not in ParentA:
                NewFrontier |= { ns }
                ParentA[ns]  = s
                if ns in ParentB:
                    if flip:
                        Result = combinePaths(ns, ParentB, ParentA)
                    else:
                        Result = combinePaths(ns, ParentA, ParentB)
                    yield Result
    Frontier.clear()
    Frontier.update(NewFrontier)

Given a `state` and a parent dictionary `Parent`, the function `path_to` returns a path leading to the given `state`.

In [ ]:
def path_to(state: State, Parent: dict[State, State]) -> list[State]:
    p = Parent[state]
    if p == state:
        return [state]
    return path_to(p, Parent) + [state]

The function `combinePath` takes three parameters:
- `state` is a state that has been reached in bidirectional BFS from both `start` and `goal`.
- `ParentA` is the parent dictionary that has been build when searching from `start`.
   If $\texttt{ParentA}[s_1] = s_2$ holds, then either $s_1 = s_2 = \texttt{start}$ or 
   $s_1 \in \texttt{next_states}(s_2)$.
- `ParentB` is the parent dictionary that has been build when searching from `goal`.
   If $\texttt{ParentB}[s_1] = s_2$ holds, then either $s_1 = s_2 = \texttt{goal}$ or
   $s_1 \in \texttt{next_states}(s_2)$.
The function returns a path from `start` to `goal`.

In [ ]:
def combinePaths(state: State, ParentA: dict[State, State], ParentB: dict[State, State]) -> list[State]:
        Path1 = path_to(state, ParentA)
        Path2 = path_to(state, ParentB)
        return Path1[:-1] + Path2[::-1] # Path2 is reversed

In [ ]:
%run Sliding-Puzzle.ipynb

In [ ]:
%load_ext memory_profiler

In [ ]:
Path: tuple[tuple[int, ...], ...]

In [ ]:
%%time
%memit Path = search(start, goal, next_states)
print(len(Path)-1)

In [ ]:
animation(Path) # type: ignore

In [ ]:
%%time
%memit Path = search(start2, goal2, next_states)
print(len(Path)-1)

In [ ]:
animation(Path) # type: ignore